#Примерная модель

In [32]:
!pip install xgboost

import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression


from sklearn import metrics
from sklearn.model_selection import train_test_split

from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
df = pd.read_csv('dataset.csv', sep='\\t')
df.head()

/usr/local/lib/python3.8/dist-packages/pandas/util/_decorators.py:311: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return func(*args, **kwargs)


,text,label
0,Предмет договора Продавец обязуется передать с...,5
1,является бывшим употреблении комплектация техн...,5
2,включая сумму НДС Указанная стоимость является...,5
3,Моментом оплаты считается дата зачисления дене...,5
4,Право собственности оборудование риск случайно...,5


In [5]:
df.dropna(inplace=True) #есть NAN значения

In [6]:
tfidf = TfidfVectorizer()
feature_matrix = tfidf.fit_transform(df['text'])
df1 = pd.DataFrame(data=feature_matrix.todense(), columns=tfidf.get_feature_names())

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [8]:
X = df1
y = df['label']

In [29]:
X.shape

(1688, 11322)

In [33]:
X_new = SelectKBest(chi2, k=100).fit_transform(X, y)
X_new.shape

(1688, 100)

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X_new, y, test_size=0.2, random_state=42)

In [35]:
estimators = [('bst', XGBClassifier(multi='softmax')), ('SGDC',SGDClassifier(random_state=42)), ('knn', KNeighborsClassifier(n_neighbors=10))]

In [36]:
clf = StackingClassifier(estimators=estimators, final_estimator=LogisticRegression())
clf.fit(X_train, y_train)

/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


StackingClassifier(estimators=[('bst', XGBClassifier(multi='softmax')),
                               ('SGDC', SGDClassifier(random_state=42)),
                               ('knn', KNeighborsClassifier(n_neighbors=10))],
                   final_estimator=LogisticRegression())

In [37]:
predicted = clf.predict(X_test)
print(metrics.classification_report(predicted, y_test))

              precision    recall  f1-score   support

           1       0.77      0.96      0.85        55
           2       0.77      0.94      0.85        68
           3       0.97      0.56      0.71       121
           4       0.85      1.00      0.92        63
           5       0.71      0.97      0.82        31

    accuracy                           0.82       338
   macro avg       0.82      0.89      0.83       338
weighted avg       0.85      0.82      0.81       338

